In [ ]:
from grindr_access.grindrUser import grindrUser
from time import sleep
import random, os, pickle
from datetime import datetime
from math import radians, cos, sin, asin, sqrt
import numpy as np
from dotenv import load_dotenv
from copy import deepcopy
from influxdb_client import InfluxDBClient, Point, WriteOptions,WritePrecision
from mongoengine import connect, NotUniqueError
load_dotenv()

user = grindrUser()
mail = os.getenv('GRINDR_MAIL')
password = os.getenv('GRINDR_PASS')
krk_lat = 50.059185
krk_lon = 19.937809

user.login(mail, password)
connect(
    db=os.getenv('MONGO_DB'),
    host=os.getenv('MONGO_URL'),
    port=27017,
    username=os.getenv('MONGO_USR'),
    password=os.getenv('MONGO_PWD'),
    authentication_source="admin"
    )


In [ ]:
from stalkr import generate_grid_points, localizeProfile

In [ ]:
# Example usage
center_lat, center_lon = krk_lat, krk_lon # Equator and Prime Meridian
side_m = 10000 # 10 km side length
accuracy_m = 1000 # 1000m 
points_per_side = int(side_m/accuracy_m) # Generate 100 points

grid_points = generate_grid_points(center_lat, center_lon, side_m, points_per_side,jitter_m=100)

In [ ]:
from profile_model import ProfileModel

In [ ]:
i = 0
profiles = {}
batch_timestamp = int(datetime.now().timestamp() * 1000)
for point in grid_points:
    timestamp = int(datetime.now().timestamp() * 1000)
    loc_date_key = (point[0], point[1], timestamp)
    time_to_sleep = random.uniform(0, 4)
    print(f"{i}/{len(grid_points)} - {point} - {timestamp} - {time_to_sleep}")
    profile_list = user.getProfiles(point[0], point[1])
    for profile in profile_list['items']:
        prof = {}
        prof.update(profile['data'])
        profile_types = ["PartialProfileV1", "FullProfileV1"]
        if any(element in prof.get('@type') for element in profile_types):
            prof['created'] = timestamp
            prof['lat'] =  point[0] # Add the lat and lon of the anchor point
            prof['lon'] =  point[1]
            prof['batch_timestamp'] = batch_timestamp
            new_doc = ProfileModel(**prof)
            new_doc.save()
            if profiles.get(new_doc.profileId) is None:
                profiles[new_doc.profileId] = []
            profiles[new_doc.profileId].append(deepcopy(prof))

    sleep(time_to_sleep)
    i+=1

In [ ]:
locations = {}
localized_profiles = {}
for profileId, profileList in profiles.items():
    new_locations, profile = localizeProfile(profileList)
    if profile:
        localized_profiles[profileId] = profile
        try:
            profile.save()  # Attempt to save the new profile
        except NotUniqueError:
            print("A document with the same unique index already exists. Ignoring.")
    locations.update(new_locations)

In [ ]:
localized_profiles[542849597]

In [ ]:
profiles[542849597]